In [1]:
# https://github.com/jarvijaakko/Apartment_hunting/blob/master/Apartment_scraper.ipynb
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from httpx import Client

/Users/tatu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
driver = webdriver.Chrome()

In [3]:
# Loop to loop through all pages
# Scrape first the number of pages (in progress)
url_list=[]
for p in range(1,4): #45
    url_01 = "https://asunnot.oikotie.fi/myytavat-asunnot?pagination="
    url_02 = str(p)
    url_03 = "&locations=%5B%22helsinki%22%5D&roomCount%5B%5D=2&cardType=100"        
    url_list.append(url_01 + url_02 + url_03)

In [4]:
# Define lists that will house the scraped results
addresses=[]
infos_a = []
infos_b = []
prices = []
sizes = []
alls = []

In [5]:
for i in url_list[0:3]:
    url = i
    driver.get(url)
    time.sleep(3)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    for a in soup.findAll('div',href=False, attrs={'class':'ot-card-v2__info-container padded padded--v20-h20'}):
        address=a.find('div', attrs={'class':'card-v2-text-container__text card-v2-text-container__text--bold'})
        info_a = a.find('p', attrs={'class': 'card-v2-text-container__text card-v2-text-container__text--wide paragraph paragraph--caption ng-star-inserted'})
        info_b = a.find('p', attrs={'class':'card-v2-text-container__text paragraph paragraph--caption ng-star-inserted'})
        price = a.find('div', attrs={'class': 'card-v2-text-container__column'})
        size = a.find('div', attrs={'class': 'card-v2-text-container__group card-v2-text-container__group--boxed'})

        print(address)
        addresses.append(address.text)
        infos_a.append(info_a.text)
        infos_b.append(info_b.text)
        prices.append(price.text)
        sizes.append(size.text)
        

<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Kalevankatu 44 A, Kamppi, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Johan Sederholmin tie 8 B, Herttoniemi, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Poutuntie 7 A, Pohjois-Haaga, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Väinämöisenkatu 21 A, Etu-Töölö, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Näyttelijäntie 4 A, Pohjois-Haaga, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-container__text--bold">Kauriintie 3 C, Siltamäki, Helsinki</div>
<div _ngcontent-ng-c3980868311="" class="card-v2-text-container__text card-v2-text-

In [6]:
df = pd.DataFrame({'Address':addresses, 'info_a':infos_a, 'info_b':infos_b, 'price':prices, 'size':sizes})
df.to_csv('Apartments.csv', index=False, encoding='utf-8') 
df = pd.read_csv('Apartments.csv')
df.head()



,Address,info_a,info_b,price,size
0,"Kalevankatu 44 A, Kamppi, Helsinki","\n\t\t\t\tKerrostalo, 1948\n\t\t\t",\n\t\t\t\t2h+avok+alkovi+kph (yhtiöjärjestykse...,\n358 000 €\n\n\n,\n\n358 000 €\n\n\n\n\n48 m²\n\n\n\n
1,"Johan Sederholmin tie 8 B, Herttoniemi, Helsinki","\n\t\t\t\tKerrostalo, 1990\n\t\t\t",\n\t\t\t\t2h+kk+s\n\t\t\t,\n204 000 €\n\n\n,"\n\n204 000 €\n\n\n\n\n47,5 m²\n\n\n\n"
2,"Poutuntie 7 A, Pohjois-Haaga, Helsinki","\n\t\t\t\tKerrostalo, 1954\n\t\t\t",\n\t\t\t\t2h+avok+kph+parveke (yhtiöjärjestyks...,\n238 000 €\n\n\n,"\n\n238 000 €\n\n\n\n\n46,6 m²\n\n\n\n"
3,"Väinämöisenkatu 21 A, Etu-Töölö, Helsinki","\n\t\t\t\tKerrostalo, 1935\n\t\t\t","\n\t\t\t\t2-3h, k, kph, vh, et\n\t\t\t",\n482 000 €\n\n\n,\n\n482 000 €\n\n\n\n\n62 m²\n\n\n\n
4,"Näyttelijäntie 4 A, Pohjois-Haaga, Helsinki","\n\t\t\t\tKerrostalo, 1954\n\t\t\t","\n\t\t\t\t2h, avok, kph, ransk.parveke\n\t\t\t",\n239 000 €\n\n\n,\n\n239 000 €\n\n\n\n\n48 m²\n\n\n\n


In [7]:

# remove escape characters from columns (ugly solution but works relatively well)
#https://stackoverflow.com/questions/8115261/how-to-remove-all-the-escape-sequences-from-a-list-of-strings
df['info_a'] = df['info_a'].replace('\n\t\t\t\t', '', regex=True). replace('\n\t\t\t', '', regex=True) # https://stackoverflow.com/questions/6116978/how-to-replace-multiple-substrings-of-a-string
df['info_b'] = df['info_b'].replace('\n\t\t\t\t', '', regex=True). replace('\n\t\t\t', '', regex=True)
df['price'] = df['price'].replace('\n', '', regex=True). replace('\n\n\n', '', regex=True)
df['size'] = df['size'].replace('\n\n', '', regex=True). replace('\n\n\n\n\n', '', regex=True).replace('\n\n\n\n', '', regex=True)
df['size'] = df['size'].replace('\n', '', regex=True) #for some reason one escape character remains and has to be deleted with an extra line of code

# some data handling
# https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns

df[['street', 'area', 'town']] = df['Address'].str.split(',', expand=True)
df[['building_type', 'year_built']] = df['info_a'].str.split(',', expand=True)

# remove price information from size column 
# https://stackoverflow.com/questions/73974777/python-remove-text-if-same-of-another-column
df['size'] = [df.loc[i, 'size']. replace(df.loc[i, 'price'], '') for i in df.index]

# split info_b into multiple columns
df['info_b'] = df['info_b'].replace('+', ',') # some columns have '+' delimiters instead of commas
df[['room_num', 'kitchen_type', 'bathroom', 'balcony', 'extra_I', 'extra_II']] = df['info_b'].str.split(',', expand=True)

# remove useless information from number of rooms column
df['room_num']=df['room_num'].str[0]

# show data frame
df.head()

,Address,info_a,info_b,price,size,street,area,town,building_type,year_built,room_num,kitchen_type,bathroom,balcony,extra_I,extra_II
0,"Kalevankatu 44 A, Kamppi, Helsinki","Kerrostalo, 1948",2h+avok+alkovi+kph (yhtiöjärjestyksen mukaan 1...,358 000 €,48 m²,Kalevankatu 44 A,Kamppi,Helsinki,Kerrostalo,1948,2,None,None,None,None,None
1,"Johan Sederholmin tie 8 B, Herttoniemi, Helsinki","Kerrostalo, 1990",2h+kk+s,204 000 €,"47,5 m²",Johan Sederholmin tie 8 B,Herttoniemi,Helsinki,Kerrostalo,1990,2,None,None,None,None,None
2,"Poutuntie 7 A, Pohjois-Haaga, Helsinki","Kerrostalo, 1954",2h+avok+kph+parveke (yhtiöjärjestyksen mukaan ...,238 000 €,"46,6 m²",Poutuntie 7 A,Pohjois-Haaga,Helsinki,Kerrostalo,1954,2,None,None,None,None,None
3,"Väinämöisenkatu 21 A, Etu-Töölö, Helsinki","Kerrostalo, 1935","2-3h, k, kph, vh, et",482 000 €,62 m²,Väinämöisenkatu 21 A,Etu-Töölö,Helsinki,Kerrostalo,1935,2,k,kph,vh,et,None
4,"Näyttelijäntie 4 A, Pohjois-Haaga, Helsinki","Kerrostalo, 1954","2h, avok, kph, ransk.parveke",239 000 €,48 m²,Näyttelijäntie 4 A,Pohjois-Haaga,Helsinki,Kerrostalo,1954,2,avok,kph,ransk.parveke,None,None


In [8]:
# remove non numeric characters from price and size columns
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular

# remove m2 character
df['size'] = df['size'].str[:2]

# https://stackoverflow.com/questions/31700691/convert-commas-decimal-separators-to-dots-within-a-dataframe
df['size'] = [x.replace(',', '.') for x in df['size']]
df['size'] = df['size'].astype(float)

# same for price
df['price'] = [x.replace('€', '') for x in df['price']]
df['price'] = df['price'].str[:7]
df['price'] = df['price'].replace('\xa0', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['price'] = df['price'].astype(float)

df.head()

,Address,info_a,info_b,price,size,street,area,town,building_type,year_built,room_num,kitchen_type,bathroom,balcony,extra_I,extra_II
0,"Kalevankatu 44 A, Kamppi, Helsinki","Kerrostalo, 1948",2h+avok+alkovi+kph (yhtiöjärjestyksen mukaan 1...,358000.0,48.0,Kalevankatu 44 A,Kamppi,Helsinki,Kerrostalo,1948,2,None,None,None,None,None
1,"Johan Sederholmin tie 8 B, Herttoniemi, Helsinki","Kerrostalo, 1990",2h+kk+s,204000.0,47.0,Johan Sederholmin tie 8 B,Herttoniemi,Helsinki,Kerrostalo,1990,2,None,None,None,None,None
2,"Poutuntie 7 A, Pohjois-Haaga, Helsinki","Kerrostalo, 1954",2h+avok+kph+parveke (yhtiöjärjestyksen mukaan ...,238000.0,46.0,Poutuntie 7 A,Pohjois-Haaga,Helsinki,Kerrostalo,1954,2,None,None,None,None,None
3,"Väinämöisenkatu 21 A, Etu-Töölö, Helsinki","Kerrostalo, 1935","2-3h, k, kph, vh, et",482000.0,62.0,Väinämöisenkatu 21 A,Etu-Töölö,Helsinki,Kerrostalo,1935,2,k,kph,vh,et,None
4,"Näyttelijäntie 4 A, Pohjois-Haaga, Helsinki","Kerrostalo, 1954","2h, avok, kph, ransk.parveke",239000.0,48.0,Näyttelijäntie 4 A,Pohjois-Haaga,Helsinki,Kerrostalo,1954,2,avok,kph,ransk.parveke,None,None


In [13]:
sub_df = df[['price', 'size', 'area', 'year_built', 'room_num']]
sub_df = sub_df.dropna(subset=['price', 'size', 'year_built'])

sub_df.head()

,price,size,area,year_built,room_num
0,358000.0,48.0,Kamppi,1948,2
1,204000.0,47.0,Herttoniemi,1990,2
2,238000.0,46.0,Pohjois-Haaga,1954,2
3,482000.0,62.0,Etu-Töölö,1935,2
4,239000.0,48.0,Pohjois-Haaga,1954,2


In [14]:
sub_df.insert(2, 'sq_price', df['price'].div(df['size'], axis=0), True)


In [15]:
# adding a column for plumbing renovation assuming it was done 50 years after building the apartment
sub_df['year_built'] = sub_df['year_built'].astype(float)
sub_df.insert(2, 'pipe_repair', df['year_built'], True)
sub_df['pipe_repair'] = sub_df['pipe_repair'].astype(int) + 50

In [16]:
# df.applymap(lambda x: 1 if x >=25 else 0)
#sub_df.insert(2, 'piperepair_plug', 
#              sub_df['pipe_repair'].applymap( lambda x: 0 if x >= 2024 else ((2023-x)/40)*1500 ),
#              True)

#sub_df['pipe_repair'].applymap( lambda x: 0 if x >= 2024 else ((2023-x)/40)*1500 )

sub_df.insert(2, 'piperepair_plug', sub_df['pipe_repair'], True)
sub_df['piperepair_plug'] = sub_df['piperepair_plug'].map(lambda x: 0 if x >= 2024 else ((2023-x)/40)*1500)
sub_df = sub_df.round(2)
sub_df.head()



,price,size,piperepair_plug,pipe_repair,sq_price,area,year_built,room_num
0,358000.0,48.0,937.5,1998,7458.33,Kamppi,1948.0,2
1,204000.0,47.0,0.0,2040,4340.43,Herttoniemi,1990.0,2
2,238000.0,46.0,712.5,2004,5173.91,Pohjois-Haaga,1954.0,2
3,482000.0,62.0,1425.0,1985,7774.19,Etu-Töölö,1935.0,2
4,239000.0,48.0,712.5,2004,4979.17,Pohjois-Haaga,1954.0,2


In [17]:
sub_df['sqprice_adjusted'] = sub_df['sq_price'] + sub_df['piperepair_plug']


In [18]:
# build a function that saves the data frame into a csv-file for further calculations done in R
sub_df.to_csv('apartments_final.csv', index=False, encoding='utf-8') 